In [1]:
from pathlib import Path

In [2]:
from secretnote.utils.version import assert_version

In [3]:
import pydantic
import ray

import secretflow
import spu

assert_version(ray, "~=2.2.0")
assert_version(pydantic, ">=1.10, <2")
assert_version(secretflow, "~=1.0")
assert_version(spu, ">=0.4.1")

In [4]:
from secretnote.compat.spu import (
    SPUClusterDef,
    SPUNode,
    SPUProtocolKind,
    SPUFieldType,
    SPURuntimeConfig,
)
from secretnote.compat.secretflow.device.driver import SFConfigSimulationFullyManaged

In [5]:
secretflow.shutdown()
secretflow.init(**SFConfigSimulationFullyManaged(parties=["alice", "bob"]).dict())

spu = secretflow.SPU(
    SPUClusterDef(
        nodes=[
            SPUNode(party="alice", address="127.0.0.1:58271"),
            SPUNode(party="bob", address="127.0.0.1:58272"),
        ],
        runtime_config=SPURuntimeConfig(
            protocol=SPUProtocolKind.SEMI2K,
            field=SPUFieldType.FM128,
        ),
    ).dict()
)

2023-09-14 17:43:26,500	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


(SPURuntime pid=8876) 2023-09-14 17:43:29.237 [info] [default_brpc_retry_policy.cc:DoRetry:52] socket error, sleep=1000000us and retry
(SPURuntime pid=8876) 2023-09-14 17:43:30.237 [info] [default_brpc_retry_policy.cc:LogHttpDetail:33] cntl ErrorCode '64', http status code '200', response header '', error msg '[E61]Fail to connect Socket{id=0 addr=127.0.0.1:58272} (0x0x120040000): Connection refused [R1][E64]Not connected to 127.0.0.1:58272 yet, server_id=0'
(SPURuntime pid=8876) 2023-09-14 17:43:30.237 [info] [default_brpc_retry_policy.cc:DoRetry:75] aggressive retry, sleep=1000000us and retry


In [6]:
alice, bob = secretflow.PYU("alice"), secretflow.PYU("bob")

(SPURuntime pid=8876) 2023-09-14 17:43:31.237 [info] [default_brpc_retry_policy.cc:LogHttpDetail:33] cntl ErrorCode '64', http status code '200', response header '', error msg '[E61]Fail to connect Socket{id=0 addr=127.0.0.1:58272} (0x0x120040000): Connection refused [R1][E64]Not connected to 127.0.0.1:58272 yet, server_id=0 [R2][E64]Not connected to 127.0.0.1:58272 yet, server_id=0'
(SPURuntime pid=8876) 2023-09-14 17:43:31.237 [info] [default_brpc_retry_policy.cc:DoRetry:75] aggressive retry, sleep=1000000us and retry
(SPURuntime pid=8877) 2023-09-14 17:43:31.293 [info] [default_brpc_retry_policy.cc:DoRetry:71] not retry for reached rcp timeout, ErrorCode '1008', error msg '[E1008]Reached timeout=2000ms @127.0.0.1:58271'


In [7]:
import jax.numpy as jnp

In [8]:
def read_numbers(filename):
    with open(filename, "r") as f:
        return jnp.array([int(item.strip()) for item in f.read().split(",")])

In [9]:
fibonacci = alice(read_numbers)(str(Path().joinpath("A000045.txt").resolve()))

(_run pid=8842) INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=8842) INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=8842) INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'
(_run pid=8842) INFO:jax._src.xla_bridge:Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this.


In [10]:
pascal = bob(read_numbers)(str(Path().joinpath("A007318.txt").resolve()))

In [11]:
def dot(x, y):
    if x.shape[0] > y.shape[0]:
        x = x[: y.shape[0]]
    else:
        y = y[: x.shape[0]]
    return jnp.dot(x, y)

In [12]:
result = spu(dot)(fibonacci.to(spu), pascal.to(spu))

INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'
INFO:jax._src.xla_bridge:Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this.


(_run pid=8843) INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=8843) INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=8843) INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'
(_run pid=8843) INFO:jax._src.xla_bridge:Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this.


In [13]:
secretflow.reveal(result)

array(-605557560, dtype=int32)